### Function Gemma - POC

Google example: https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/functiongemma/finetuning-with-functiongemma.ipynb#scrollTo=2d4f0a33

In [1]:
! pip install -q transformers==4.57.3 datasets accelerate evaluate trl==0.26.2 protobuf sentencepiece
! pip install -q huggingface_hub tensorboard


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
token = os.getenv("HF_TOKEN")

In [4]:
from transformers.utils import get_json_schema

# TOOLS internas

def pagamento_pix(
    valor: float,
    nome_estabelecimento: str
) -> str:
    """Realiza um pagamento via Pix na maquininha.

    Args:
        valor: Valor do pagamento em reais (BRL)
        nome_estabelecimento: Nome exibido na maquininha
    """
    return "Pagamento Pix realizado"


def imprimir_comprovante(
    tipo: str
) -> str:
    """Imprime o comprovante do pagamento.

    Args:
        tipo: Tipo do comprovante ("cliente" ou "estabelecimento")
    """
    return "Comprovante impresso"


# Macro tool

def pagamento(
    valor: float,
    nome_estabelecimento: str,
    imprimir: bool = True
) -> str:
    """
    Macro tool de pagamento.
    Realiza pagamento via Pix e imprime comprovante.

    Args:
        valor: Valor do pagamento em reais (BRL)
        nome_estabelecimento: Nome exibido na maquininha
        imprimir: Se deve imprimir comprovante
    """

    resultado_pagamento = pagamento_pix(
        valor=valor,
        nome_estabelecimento=nome_estabelecimento
    )

    if imprimir:
        imprimir_comprovante(tipo="cliente")

    return resultado_pagamento


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/functiongemma-270m-it", device_map="auto", token=token)
model = AutoModelForCausalLM.from_pretrained("google/functiongemma-270m-it", dtype="auto", device_map="auto", token=token)

In [5]:
import json
from datasets import Dataset

# FunctionGemma special tokens 
START_TURN = ""
END_TURN = ""
START_DECL = ""
END_DECL = ""
START_CALL = ""
END_CALL = ""
ESCAPE = ""

FUNCTION_DECLARATIONS = f"""{START_DECL}declaration:pagamento{{description:{ESCAPE}Realiza um pagamento via Pix na maquininha e opcionalmente imprime o comprovante{ESCAPE},parameters:{{properties:{{valor:{{description:{ESCAPE}Valor do pagamento em reais (BRL){ESCAPE},type:{ESCAPE}NUMBER{ESCAPE}}},nome_estabelecimento:{{description:{ESCAPE}Nome exibido na maquininha{ESCAPE},type:{ESCAPE}STRING{ESCAPE}}},imprimir:{{description:{ESCAPE}Indica se deve imprimir o comprovante{ESCAPE},type:{ESCAPE}BOOLEAN{ESCAPE}}}}},required:[{ESCAPE}valor{ESCAPE},{ESCAPE}nome_estabelecimento{ESCAPE}],type:{ESCAPE}OBJECT{ESCAPE}}}}}{END_DECL}"""

SYSTEM_PROMPT = f"""{START_TURN}developer
Você é um modelo especialista em **chamada de funções (function calling)** para uma **maquininha de pagamento (POS)**.

Seu papel é:
- Interpretar a solicitação do usuário
- Identificar a função correta a ser chamada
- Retornar **exclusivamente** uma chamada de função válida, no formato especificado

As funções disponíveis estão descritas abaixo:
{FUNCTION_DECLARATIONS}

Regras importantes:
- Chame **apenas** funções listadas acima
- Preencha corretamente todos os parâmetros obrigatórios
- Use somente os tipos definidos no schema
- Não gere texto explicativo fora da chamada de função
- Se a solicitação **não corresponder** a nenhuma função disponível, retorne exatamente:
  "Não especializado para esse tipo de ação"

{END_TURN}
"""


In [6]:
def create_training_example(sample):
    """
    Input sample:
    {
      "user_content": "pix de 50 reais sem comprovante",
      "tool_name": "pagamento",
      "tool_arguments": "{\"valor\":50,\"nome_estabelecimento\":\"Padaria Central\",\"imprimir\":false}"
    }
    """

    user_content = sample["user_content"]
    tool_name = sample["tool_name"]
    tool_args = json.loads(sample["tool_arguments"])

    # Prompt (entrada)
    prompt = f"""{SYSTEM_PROMPT}{START_TURN}user
{user_content}
{END_TURN}
{START_TURN}model
"""

    # Completion (chamada da função)
    params_str = ",".join(
        [f"{k}:{ESCAPE}{v}{ESCAPE}" for k, v in tool_args.items()]
    )

    completion = f"{START_CALL}call:{tool_name}{{{params_str}}}{END_CALL}"

    return {"text": prompt + completion}


In [8]:
# =============================================================================
# Load and convert dataset
# =============================================================================
raw_data = []
with open('machine_actions.jsonl', 'r') as f:
    for line in f:
        raw_data.append(json.loads(line.strip()))

print(f"Loaded {len(raw_data)} raw examples")

dataset = Dataset.from_list(raw_data)
dataset = dataset.map(create_training_example, remove_columns=dataset.features)

# Split into train/test (90%/10%)
dataset = dataset.train_test_split(test_size=0.5, shuffle=True, seed=42)

print(f"\nDataset prepared:")
print(f"   Train: {len(dataset['train'])} examples")
print(f"   Test:  {len(dataset['test'])} examples")

# Show sample
print(f"\n{'='*60}")
print("Sample training example:")
print("="*60)
print(dataset['train'][0]['text'][:800])
print("...")
     

Loaded 114 raw examples


Map: 100%|██████████| 114/114 [00:00<00:00, 10856.20 examples/s]


Dataset prepared:
   Train: 57 examples
   Test:  57 examples

Sample training example:
developer
Você é um modelo especialista em **chamada de funções (function calling)** para uma **maquininha de pagamento (POS)**.

Seu papel é:
- Interpretar a solicitação do usuário
- Identificar a função correta a ser chamada
- Retornar **exclusivamente** uma chamada de função válida, no formato especificado

As funções disponíveis estão descritas abaixo:
declaration:pagamento{description:Realiza um pagamento via Pix na maquininha e opcionalmente imprime o comprovante,parameters:{properties:{valor:{description:Valor do pagamento em reais (BRL),type:NUMBER},nome_estabelecimento:{description:Nome exibido na maquininha,type:STRING},imprimir:{description:Indica se deve imprimir o comprovante,type:BOOLEAN}},required:[valor,nome_estabelecimento],type:OBJECT}}

Regras importantes:
- Chame **apen
...


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# =============================================================================
# Load FunctionGemma base model
# =============================================================================
BASE_MODEL = "google/functiongemma-270m-it"

print(f"Loading {BASE_MODEL}...")
print("   (Downloads ~540MB on first run, then uses cache)")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,      # 16-bit to save VRAM
    device_map="auto",                # Automatically load to GPU
    attn_implementation="eager"       # Without FlashAttention for compatibility
)

# Tokenizer converts text to tokens and back
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print(f"\nModel loaded!")
print(f"   Parameters: {model.num_parameters():,}")
print(f"   Memory: ~{model.num_parameters() * 2 / 1e9:.1f} GB (bfloat16)")
print(f"   Device: {model.device}")

Loading google/functiongemma-270m-it...
   (Downloads ~540MB on first run, then uses cache)


`torch_dtype` is deprecated! Use `dtype` instead!



Model loaded!
   Parameters: 268,098,176
   Memory: ~0.5 GB (bfloat16)
   Device: cuda:0


In [10]:
# parameters
checkpoint_dir = f"/content-function-gemma"
learning_rate = 5e-5 #@param {type:"number"}

In [12]:
from trl import SFTConfig
import torch

torch_dtype = model.dtype

args = SFTConfig(
    output_dir=checkpoint_dir,              # directory to save and repository id
    max_length=512,                         # max sequence length for model and packing of the dataset
    packing=False,                          # Groups multiple samples in the dataset into a single sequence
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=4,          # batch size per device during training
    gradient_checkpointing=False,           # Caching is incompatible with gradient checkpointing
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=1,                        # log every step
    #save_strategy="epoch",                  # save checkpoint every epoch
    eval_strategy="epoch",                  # evaluate checkpoint every epoch
    learning_rate=learning_rate,            # learning rate
    fp16=True if torch_dtype == torch.float16 else False,   # use float16 precision
    bf16=True if torch_dtype == torch.bfloat16 else False,  # use bfloat16 precision
    lr_scheduler_type="constant",            # use constant learning rate scheduler
    push_to_hub=True,                        # push model to hub
    report_to="tensorboard",                 # report metrics to tensorboard
)

In [13]:
from trl import SFTTrainer

# Create Trainer object
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

Truncating eval dataset: 100%|██████████| 57/57 [00:00<?, ? examples/s]
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [14]:
# Start training, the model will be automatically saved to the Hub and the output directory
trainer.train()

# Save the final model again to the Hugging Face Hub
trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,0.119500,0.102062,0.141911,17376.000000,0.977363
2,0.066500,0.094530,0.080961,34752.000000,0.976176
3,0.039100,0.081038,0.055808,52128.000000,0.978748


Processing Files (5 / 5): 100%|██████████|  574MB /  574MB,  261MB/s  
New Data Upload: 100%|██████████| 9.78kB / 9.78kB, 5.43kB/s  


In [15]:
FINAL_MODEL_DIR = f"model-tuned-final"

# Save model weights and config
trainer.save_model(FINAL_MODEL_DIR)

# Save tokenizer (needed for inference)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print(f"Model saved locally to {FINAL_MODEL_DIR}/")

Processing Files (5 / 5): 100%|██████████|  574MB /  574MB,  274MB/s  
New Data Upload: |          |  0.00B /  0.00B,  0.00B/s  
No files have been modified since last commit. Skipping to prevent empty commit.


Model saved locally to model-tuned-final/


In [18]:
# =============================================================================
# Testar o modelo fine-tuned com novos prompts 
# =============================================================================
# CRÍTICO: usar exatamente o mesmo formato Google FunctionGemma do treino

test_prompts = [
    "quero pagar 50 reais no pix",
    "faz um pix de 30 reais",
    "pix de 100 reais sem comprovante",
    "pagar 25 reais via pix",
    "realiza um pagamento pix de 80 reais sem imprimir",
]

print("Testando modelo fine-tuned")
print("=" * 60)

for prompt in test_prompts:
    # Prompt no MESMO formato do treino
    input_text = f"""{SYSTEM_PROMPT}{START_TURN}user
{prompt}
{END_TURN}
{START_TURN}model
"""
    
    # Tokenizar e enviar para GPU
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    # Gerar resposta
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id
    )
    
    # Decodificar apenas tokens novos
    response = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=False
    )
    
    print(f"\nUsuário: {prompt}")
    print(f"Modelo: {response.strip()}")
    
    print("-" * 60)


Testando modelo fine-tuned

Usuário: quero pagar 50 reais no pix
Modelo: call:pagamento{valor:50,nome_estabelecimento:Mercado Verde,imprimir:True}<eos>
------------------------------------------------------------

Usuário: faz um pix de 30 reais
Modelo: call:pagamento{valor:30,nome_estabelecimento:Mercado Verde,imprimir:True}<eos>
------------------------------------------------------------

Usuário: pix de 100 reais sem comprovante
Modelo: call:pagamento{valor:100,nome_estabelecimento:Hotel Central,imprimir:False}<eos>
------------------------------------------------------------

Usuário: pagar 25 reais via pix
Modelo: call:pagamento{valor:25,nome_estabelecimento:Hotel Central,imprimir:True}<eos>
------------------------------------------------------------

Usuário: realiza um pagamento pix de 80 reais sem imprimir
Modelo: call:pagamento{valor:80,nome_estabelecimento:Hotel Central,imprimir:False}<eos>
------------------------------------------------------------
